In [1]:
import cv2, time, os, dlib, math
import PIL.Image, PIL.ImageDraw

import face_recognition
import pygame
import random
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
smile_cascade = cv2.CascadeClassifier("haarcascade_smile.xml")
leftEye_cascade = cv2.CascadeClassifier("haarcascade_lefteye_2splits.xml")
rightEye_cascade = cv2.CascadeClassifier("haarcascade_righteye_2splits.xml")

hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:

##############################################################333
###############################################################zfadfad

    
    
pygame.init
pygame.font.init()
pygame.mixer.init()

WIDTH, HEIGHT = 1280, 720
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("ALIEN MIMICS")


#GLOBAL VARIABLES
i = 0
punkty = 0


# Load images
RED_SPACE_SHIP = pygame.image.load(os.path.join("assets", "red1.png"))
GREEN_SPACE_SHIP = pygame.image.load(os.path.join("assets", "green1.png"))
BLUE_SPACE_SHIP = pygame.image.load(os.path.join("assets", "blue.png"))


# Player player
YELLOW_SPACE_SHIP = pygame.image.load(os.path.join("assets", "ship.png"))


# Lasers
RED_LASER = pygame.image.load(os.path.join("assets", "pixel_laser_red.png"))
GREEN_LASER = pygame.image.load(os.path.join("assets", "pixel_laser_green.png"))
BLUE_LASER = pygame.image.load(os.path.join("assets", "pixel_laser_blue.png"))
YELLOW_LASER = pygame.image.load(os.path.join("assets", "bulletgg.png"))


# Sounds and music
player_laser = pygame.mixer.Sound(os.path.join("sound", "laser1.wav"))
enemy_laser = pygame.mixer.Sound(os.path.join("sound", "laser2.wav"))
player_death = pygame.mixer.Sound(os.path.join("sound", "player_death.wav"))
enemy_death = pygame.mixer.Sound(os.path.join("sound", "enemy_death.wav"))

player_laser.set_volume(0.5)
enemy_laser.set_volume(0.5)
player_death.set_volume(0.5)
enemy_death.set_volume(1.0)

BG_music = pygame.mixer.music.load(os.path.join("sound", "bg_music.mp3"))


# Play Music
pygame.mixer.music.play(-1)


# Background
BG = [pygame.transform.scale(pygame.image.load(os.path.join("assets", "1.png")), (WIDTH, HEIGHT)), 
      pygame.transform.scale(pygame.image.load(os.path.join("assets", "2.png")), (WIDTH, HEIGHT)), 
      pygame.transform.scale(pygame.image.load(os.path.join("assets", "3.png")), (WIDTH, HEIGHT)), 
      pygame.transform.scale(pygame.image.load(os.path.join("assets", "4.png")), (WIDTH, HEIGHT)),
      pygame.transform.scale(pygame.image.load(os.path.join("assets", "5.png")), (WIDTH, HEIGHT)),
      pygame.transform.scale(pygame.image.load(os.path.join("assets", "6.png")), (WIDTH, HEIGHT))]
menu_bg = pygame.transform.scale(pygame.image.load(os.path.join("assets", "menu_bg.jpg")), (WIDTH, HEIGHT))




def odl(x1, y1, x2, y2): 
    roz_y = y1 - y2
    roz_x = x1 - x2
    return int(math.sqrt( roz_x * roz_x + roz_y * roz_y ))





def naj_wynik():
    with open("highscore.txt", "r") as f:
        return f.read()
    
  
    
       

class Laser:
    def __init__(self, x, y, img):
        self.x = x
        self.y = y
        self.img = img
        self.mask = pygame.mask.from_surface(self.img)

    def draw(self, window):
        window.blit(self.img, (self.x, self.y))

    def move(self, vel):
        self.y += vel

    def off_screen(self, height):
        return not(self.y <= height and self.y >= 0)

    def zderzenie(self, obj):
        return kolizja(self, obj)

    
# KLASA MATKA dla player i enemy
class Ship:
    COOLDOWN = 10

    def __init__(self, x, y, zdrowie=100):
        self.x = x
        self.y = y
        self.zdrowie = zdrowie
        self.ship_img = None
        self.laser_img = None
        self.lasers = []
        self.cool_down_counter = 0

    def draw(self, window):
        window.blit(self.ship_img, (self.x, self.y))
        for laser in self.lasers:
            laser.draw(window)

    def move_lasers(self, vel, obj):
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            elif laser.zderzenie(obj):
                obj.zdrowie -= 10
                self.lasers.remove(laser)

    def cooldown(self):
        if self.cool_down_counter >= self.COOLDOWN:
            self.cool_down_counter = 0
        elif self.cool_down_counter > 0:
            self.cool_down_counter += 1

    def strzal(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x+25, self.y-30, self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter = 1
            player_laser.play()

    def get_width(self):
        return self.ship_img.get_width()

    def get_height(self):
        return self.ship_img.get_height()


class Player(Ship):
    global punkty
    def __init__(self, x, y, zdrowie=100):
        super().__init__(x, y, zdrowie)
        self.ship_img = YELLOW_SPACE_SHIP
        self.laser_img = YELLOW_LASER
        self.mask = pygame.mask.from_surface(self.ship_img)
        self.max_zdrowie = zdrowie

    def move_lasers(self, vel, objs):
        global punkty
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            else:
                for obj in objs:
                    if laser.zderzenie(obj):
                        objs.remove(obj)
                        punkty += 10
                        enemy_death.play()
                        if laser in self.lasers:
                            self.lasers.remove(laser)

    def draw(self, window):
        super().draw(window)
        self.zdrowiebar(window)

    def zdrowiebar(self, window):
        pygame.draw.rect(window, (255,0,0), (self.x, self.y + self.ship_img.get_height() + 10, self.ship_img.get_width(), 10))
        pygame.draw.rect(window, (0,255,0), (self.x, self.y + self.ship_img.get_height() + 10, self.ship_img.get_width() * (self.zdrowie/self.max_zdrowie), 10))


class Enemy(Ship):
    COLOR_MAP = {
                "red": (RED_SPACE_SHIP, RED_LASER),
                "green": (GREEN_SPACE_SHIP, GREEN_LASER),
                "blue": (BLUE_SPACE_SHIP, BLUE_LASER)
                }

    def __init__(self, x, y, color, zdrowie=100):
        super().__init__(x, y, zdrowie)
        self.ship_img, self.laser_img = self.COLOR_MAP[color]
        self.mask = pygame.mask.from_surface(self.ship_img)

    def move(self, vel):
        self.y += vel

    def strzal(self):
        if self.cool_down_counter == 0:
            laser = Laser(self.x-15, self.y, self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter = 1
            if self.y > 0:
                enemy_laser.play()


def kolizja(obj1, obj2):
    offset_x = obj2.x - obj1.x
    offset_y = obj2.y - obj1.y
    return obj1.mask.overlap(obj2.mask, (offset_x, offset_y)) != None

def main():
    global punkty
    run = True
    FPS = 60
    poziom = 0
    szanse = 15
    
    play_death_sound = 0
    main_font = pygame.font.SysFont("chiller", 50)
    lost_font = pygame.font.SysFont("chiller", 70)
    
    
    video=cv2.VideoCapture(1)
    
    enemies = []
    wave_length = 5
    enemy_vel = 3

    player_vel = 18
    laser_vel = 15

    player = Player(640, 630)

    clock = pygame.time.Clock()

    lost = False
    lost_count = 0

    def redraw_window():
        #WIN.blit(BG, (0,0))
        global i
        global punkty
        
        
        i+=1
        if i == 6:
            i = 0
        WIN.blit(BG[i], (0,0))
        # draw text
        szanse_label = main_font.render(f"Szanse: {szanse}", 1, (150,255,255))
        poziom_label = main_font.render(f"Poziom: {poziom}", 1, (150,255,255))
        punkty_label = main_font.render(f"Punkty: {punkty}", 1, (150,255,255))
        
        WIN.blit(szanse_label, (10, 10))
        WIN.blit(poziom_label, (WIDTH - poziom_label.get_width() - 10, 10))
        WIN.blit(punkty_label, (10, 720-punkty_label.get_height() - 10))
        

        for enemy in enemies:
            enemy.draw(WIN)

        player.draw(WIN)

        if lost:
            try:
                if (punkty > int(naj_wynik())):
                    with open("highscore.txt", "w") as f:
                        f.write(str(punkty))
                wynik = int(naj_wynik())
            except:
                wynik = 0
                
            lost_label = lost_font.render("Koniec Gry!", 1, (150,255,255))
            highpunkty_label = lost_font.render(f"Aktualny Highscore: {wynik}", 1, (150,255,255))
            WIN.blit(lost_label, (WIDTH/2 - lost_label.get_width()/2, 220))
            WIN.blit(highpunkty_label, (WIDTH/2 - highpunkty_label.get_width()/2, 300))
            
            

        pygame.display.update()

    while run:
        clock.tick(FPS)
        redraw_window()

        
        
        
        
        ##############################
        #        START FACE DETECTION
        ###############################
        
        #1 to kamera custom 0 to laptop

        check,frame=video.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

  
      

    #PRESS Q TO EXIT TEXT
        cv2.putText(frame,'Press "Q" to exit',(10, 20), fontScale=0.5,
                fontFace=cv2.FONT_HERSHEY_COMPLEX, color=(255,255,255))


     #FACE DET
        face = face_cascade.detectMultiScale(gray,scaleFactor=2.0,minNeighbors=5)  
        for x,y,w,h in face:
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)



    #CONVERT FACE TO GRAY
            the_face = frame[y:y+h, x:x+w]
            right_face = frame[y:y+h, x:x+int((w/2))]
            left_face = frame[y:y+h, x+int((w/2)):x+w]

            face_gray = 0
            face_gray = cv2.cvtColor(the_face,cv2.COLOR_BGR2GRAY)
            left_face_gray = cv2.cvtColor(left_face,cv2.COLOR_BGR2GRAY)
            right_face_gray = cv2.cvtColor(right_face,cv2.COLOR_BGR2GRAY)


             #######################################
             ### DLIB DETECT FACE LANDMARKS######
            faces = hog_face_detector(gray)
            for f in faces:
                dlib_landmarks = dlib_facelandmark(gray, f)
            ################################################  
            
            
    #SMILE DET    
            smile = smile_cascade.detectMultiScale(face_gray,scaleFactor=1.7,minNeighbors=35)    
            for x_, y_, w_, h_ in smile:
                img = cv2.rectangle(the_face,(x_,y_),(x_+w_,y_+h_),(255,0,0),3)


    #PRINT "SMILING", "eyebrow detect"
            if len(smile) > 0:
                cv2.putText(frame,'Smiling',(x, y+h+40), fontScale=0.7,
                fontFace=cv2.FONT_HERSHEY_COMPLEX, color=(255,255,255))
                player.strzal()
                
            if odl(dlib_landmarks.part(20).x, dlib_landmarks.part(20).y, dlib_landmarks.part(40).x, dlib_landmarks.part(40).y) > 32 and player.y - player_vel > 0:
                cv2.putText(frame,'Eyebrows up',(x, y+h+70), fontScale=0.7,
                fontFace=cv2.FONT_HERSHEY_COMPLEX, color=(255,255,255))
                player.y -= player_vel
                
            if odl(dlib_landmarks.part(20).x, dlib_landmarks.part(20).y, dlib_landmarks.part(40).x, dlib_landmarks.part(40).y) <= 18 and player.y + player_vel + player.get_height() + 15 < HEIGHT:
                cv2.putText(frame,'Eyebrows down',(x, y+h+70), fontScale=0.7,
                fontFace=cv2.FONT_HERSHEY_COMPLEX, color=(255,255,255))
                player.y += player_vel



    #RIGHT EYE DET     
            rightEye = rightEye_cascade.detectMultiScale(right_face_gray,scaleFactor=1.3, minNeighbors=24)        
            for x__,y__,w__,h__ in rightEye:
                img = cv2.rectangle(right_face,(x__,y__),(x__+w__,y__+h__),(0,0,0),3)

            if len(rightEye) == 0 and player.x + player_vel + player.get_width() < WIDTH: # right
            
                cv2.putText(frame,'Right Closed',(x-40, y-20), fontScale=0.5,
                fontFace=cv2.FONT_HERSHEY_COMPLEX, color=(255,255,255))
                player.x += player_vel


    #LEFT EYE DET
            leftEye = leftEye_cascade.detectMultiScale(left_face_gray,scaleFactor=1.3,minNeighbors=14)        
            for x___,y___,w___,h___ in leftEye:
                img = cv2.rectangle(left_face,(x___,y___),(x___+w___,y___+h___),(0,0,255),3)

            if len(leftEye) == 0 and player.x - player_vel > 0: # left
                
                cv2.putText(frame,'Left Closed',(x+w-40, y-20), fontScale=0.5,
                fontFace=cv2.FONT_HERSHEY_COMPLEX, color=(255,255,255))
                player.x -= player_vel
     
        cv2.imshow('Detection',frame)

        
        
        
        
        ##############################
        #        STOP FACE DETECTION
        ###############################
        
        
        
        
        
        

        
        
        if szanse <= 0 or player.zdrowie <= 0:
            lost = True
            lost_count += 1
            play_death_sound += 1
            if play_death_sound == 1:
                player_death.play()
                play_death_sound += 1

        if lost:
            if lost_count > FPS * 3:
                video.release()
                play_death_sound = 0
                punkty = 0
                run = False
            else:
                continue

        if len(enemies) == 0:
            poziom += 1
            wave_length += 3
            for i in range(wave_length):
                enemy = Enemy(random.randrange(50, WIDTH-100), random.randrange(-1000, -50), random.choice(["red", "blue", "green"]))
                enemies.append(enemy)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quit()

        klawisz = pygame.key.get_pressed()
        if klawisz[pygame.K_a] and player.x - player_vel > 0: # left
            player.x -= player_vel
        if klawisz[pygame.K_d] and player.x + player_vel + player.get_width() < WIDTH: # right
            player.x += player_vel
        if klawisz[pygame.K_w] and player.y - player_vel > 0: # up
            player.y -= player_vel
        if klawisz[pygame.K_s] and player.y + player_vel + player.get_height() + 15 < HEIGHT: # down
            player.y += player_vel
        if klawisz[pygame.K_SPACE]:
            player.strzal()
            
        if klawisz[pygame.K_q]:
            pygame.quit()
            video.release()
            cv2.destroyAllWindows()
            
            

        for enemy in enemies[:]:
            enemy.move(enemy_vel)
            enemy.move_lasers(laser_vel, player)

            if random.randrange(0, 2*60) == 1:
                enemy.strzal()

            if kolizja(enemy, player):
                player.zdrowie -= 10
                enemies.remove(enemy)
                enemy_death.play()
                punkty += 5
            elif enemy.y + enemy.get_height() > HEIGHT:
                szanse -= 1
                enemies.remove(enemy)
                punkty -= 10

        player.move_lasers(-laser_vel, enemies)

        
        
        
################ MAIN MENU #####################       
def main_menu(): 
    title_font = pygame.font.SysFont("chiller", 120)
    run = True
    while run:
        WIN.blit(menu_bg, (0,0))
        title_label = title_font.render("Kliknij LPM", 1, (0,0,0))
        WIN.blit(title_label, (WIDTH/2 - title_label.get_width()/2, 170))
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                main()
    pygame.quit()


main_menu()


error: display Surface quit